In [11]:
import pandas as pd
import random
import uuid
from datetime import datetime, timedelta
from tqdm import tqdm
import os

In [12]:
data_folder = os.path.join(os.getcwd(), os.pardir, 'data')
db_path = os.path.join(data_folder, 'retail_chain.db')
pos_path = os.path.join(data_folder, 'point_of_sales_data.csv')
customer_path = os.path.join(data_folder, 'customer_data.csv')
product_path = os.path.join(data_folder, 'product_data.csv')

In [2]:
# Configuration
# Store and Product Details
# Probability distribution for realistic trends
# Random inconsistencies parameters
# Generate Data
    # Point of Sales Data
        # Add date format inconsistencies
        # Apply category distribution
        # Apply payment distribution
        # Add duplicates
        # Create DataFrame
    # Customer Data
        # Apply age distribution
# Save to CSV

In [3]:
# Configuration
num_rows = random.randint(1_400_000, 1_600_000)
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 12, 31)

In [4]:
# Store and Product Details
stores = [f'Store_{i}' for i in range(1, 51)] + ['Online']
categories = ['Groceries','Electronics','Clothing','Home Goods']
payment_methods = ['Credit Card','Debit Card','Cash','Digital Wallet']

In [5]:
# Probability distribution for realistic trends
category_distribution = {'Groceries': 0.5, 'Electronics': 0.2, 'Clothing': 0.15, 'Home Goods': 0.15}
payment_distribution = {'Credit Card': 0.4, 'Debit Card': 0.2, 'Cash': 0.3, 'Digital Wallet': 0.1}
gender_distribution_u35 = {'M': 0.475, 'F': 0.475, 'Other': 0.05}
gender_distribution_o35 = {'M': 0.49, 'F': 0.49, 'Other': 0.01}
age_distribution = {}
for i in range(16, 75):
    if i < 18:
        age_distribution[i] = 0.05/(18-16)
    elif i < 25:
        age_distribution[i] = 0.2/(25-18)
    elif i < 35:
        age_distribution[i] = 0.25/(35-25)
    elif i < 45:
        age_distribution[i] = 0.20/(45-35)
    elif i < 55:
        age_distribution[i] = 0.18/(55-45)
    elif i < 65:
        age_distribution[i] = 0.08/(65-55)
    elif i <= 75:
        age_distribution[i] = 0.04/(75-65)

months = [1,2,3,4,5,6,7,8,9,10,11,12]
random_month_weights = {month: random.random() for month in months}
total_month_weight = sum(random_month_weights.values())
normalised_month_weights = {month: weight / total_month_weight for month, weight in random_month_weights.items()}
random_store_weights = {store: random.random() for store in stores}
total_store_weight = sum(random_store_weights.values())
normalised_store_weights = {store: weight / total_store_weight for store, weight in random_store_weights.items()}

In [6]:
# Random inconsistencies parameters
duplicate_ratio = 0.005 # 0.5%
missing_ratio = 0.01 # 1%

In [7]:
# Point of Sales Data
pos_data = []
product_data = {}
for _ in tqdm(range(num_rows), desc="Generating Data", unit="row", ncols=100):
    transaction_id = str(uuid.uuid4())
    customer_id = f'CUST_{random.randint(1000, 999999)}'
    store_id = random.choices(list(normalised_store_weights.keys()), weights=normalised_store_weights.values())[0]
    
    year = 2024
    month = random.choices(list(normalised_month_weights.keys()), weights=normalised_month_weights.values())[0]
    day_of_month = random.randint(1, 31) if month in [1,3,5,8,10,12] else random.randint(1,28)
    

    date = datetime(year, int(month), day_of_month)

    # Add date format inconsistencies
    if random.random() < 0.02:
        date_str = date.strftime('%Y/%m/%d') if random.random() < 0.5 else date.strftime('%d-%m-%Y')
    else:
        date_str = date.strftime('%Y-%m-%d')

    # Apply category distribution
    category = random.choices(list(category_distribution.keys()), weights=category_distribution.values())[0]
    product_name = f'{category}_Product_{random.randint(1, 200)}'
    quantity = random.randint(1, 10)


    # Apply payment distribution
    payment_method = random.choices(list(payment_distribution.keys()), weights=payment_distribution.values())[0]
    discount_applied = random.choice([0,5,10,15,20]) if random.random() < 0.3 else 0

    online_order = 1 if store_id == 'Online' else 0

    pos_row = [
        transaction_id, customer_id, store_id, date_str, category,
        product_name, quantity, payment_method, discount_applied, online_order
    ]
    pos_data.append(pos_row)
    if product_name not in product_data:
        product_data[product_name] = round(random.uniform(2.0, 100.0), 2)


# Add duplicates
num_duplicates = int(num_rows * duplicate_ratio)
pos_data.extend(random.choices(pos_data, k=num_duplicates))

# Create DataFrame
pos_columns = [
    'transaction_id', 'customer_id', 'store_id', 'date', 'category', 
    'product_name', 'quantity', 'payment_method', 'discount_applied', 'online_order']

pos_df = pd.DataFrame(pos_data, columns=pos_columns)
product_df = pd.DataFrame(product_data.items(), columns=['product_name', 'unit_price'])

Generating Data: 100%|████████████████████████████████| 1500083/1500083 [00:28<00:00, 52413.93row/s]


In [8]:
product_df['category'] = product_df['product_name'].apply(lambda x: x.split('_')[0])

In [9]:
# Customer Data
customer_data = []
unique_customers = pos_df['customer_id'].unique()

for customer_id in tqdm(unique_customers, desc="Generating Data", unit="row", ncols=100):
    # Apply age distribution
    customer_age = random.choices(list(age_distribution.keys()), weights=age_distribution.values())[0] if random.random() > missing_ratio else None
    if customer_age != None:
        if customer_age < 35 and random.random() > missing_ratio:
            customer_gender = random.choices(list(gender_distribution_u35.keys()), weights=gender_distribution_u35.values())[0] 
        elif customer_age > 35 and random.random() > missing_ratio:
            customer_gender = random.choices(list(gender_distribution_o35.keys()), weights=gender_distribution_o35.values())[0]
        else:
            customer_gender = None
    else:
        customer_gender = random.choices(['M','F','Other'], weights=[0.4825,0.4825,0.035])[0]
    row = [customer_id, customer_age, customer_gender]
    customer_data.append(row)

cust_columns = ['customer_id', 'age', 'gender']
cust_df = pd.DataFrame(customer_data, columns=cust_columns)


Generating Data: 100%|█████████████████████████████████| 776588/776588 [00:03<00:00, 195519.41row/s]


In [13]:
pos_df.to_csv(pos_path, index=False)
cust_df.to_csv(customer_path, index=False)
product_df.to_csv(product_path, index=False)